In [13]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from helpers import helpers_formulate_lanes as formulate_lanes


Q1 Make Trackbar for the following images to tune the parameters of canny
detection.

In [14]:
def CallBack(Copy_x):
    pass
cv2.namedWindow('output1')
cv2.namedWindow('output2')
cv2.namedWindow('Trackbars')
cv2.createTrackbar('low_trsld','Trackbars',0,400,CallBack)
cv2.createTrackbar('high_trsld','Trackbars',0,400,CallBack)
def canny_detection(img):
    blurred = cv2.GaussianBlur(img,(7,7),0)
    return cv2.Canny(blurred,cv2.getTrackbarPos('low_trsld','Trackbars'),cv2.getTrackbarPos('high_trsld','Trackbars'))

while True:
    input1 = cv2.imread('solidWhiteCurve.jpg' )
    input2 = cv2.imread('solidYellowCurve2.jpg')
    #converting 2 grayscale
    gray_input1=cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
    gray_input2=cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

    edges1 = canny_detection(gray_input1)
    edges2 = canny_detection(gray_input2)

    cv2.imshow('output1',edges1)
    cv2.imshow('output2',edges2)
    k = cv2.waitKey(100)
    if k == ord('q'):
        break
cv2.destroyAllWindows()

observation: varying the lower threshold and upper threshold reduces unwanted noise
optimum values for this 2 pics low: nearly 245 , high :210
we used some blurring to remove noise 

Q2 The out of question 1 make it as input for question 2 after make region of
interest then make trackbar for houghlines parameters (threshold ,min_line_length
max_line_gap
and then draw 2 line on image

In [16]:
#edges1 , edges2 are my new inputs
def maskROI(img):
  mask = np.zeros_like(img)
  if len(img.shape) > 2:
    channel_count = img.shape[2]
    ignore_mask_color = (255,) * channel_count
  else:
    ignore_mask_color = 255
  imshape = img.shape        
  vertices = np.array([
    [
      ((1/6*imshape[1]), imshape[0]),
      ((5/12*imshape[1]), (3/5*imshape[0])),
      ((7/12*imshape[1]), (3/5*imshape[0])),
      ((9/10*imshape[1]), imshape[0])
    ]], dtype=np.int32)
  cv2.fillPoly(mask, vertices, ignore_mask_color)
  masked_edges = cv2.bitwise_and(img, mask)
  return(masked_edges)
  
def canny_detection(img):
    blurred = cv2.GaussianBlur(img,(5,5),0)
    return cv2.Canny(blurred,248,210)

def draw_lines(lines, masked_edges):
    color = [243, 105, 14]
    thickness = 12
    lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    for line in lines:
      for x1,y1,x2,y2 in line:
        cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
    return(lines_image)

cv2.namedWindow('Trackbars')
cv2.namedWindow('output1')
cv2.namedWindow('output2')
cv2.createTrackbar('Threshold','Trackbars',10,50,CallBack)
cv2.createTrackbar('MIN_LINE_LEN','Trackbars',1,100,CallBack)
cv2.createTrackbar('MAX_LINE_GAP','Trackbars',1,100,CallBack)

RHO = 1                   
THETA = np.pi/180       

while True:
  input1 = cv2.imread('solidWhiteCurve.jpg' )
  input2 = cv2.imread('solidYellowCurve2.jpg')
  #converting 2 grayscale
  gray_input1=cv2.cvtColor(input1, cv2.COLOR_BGR2GRAY)
  gray_input2=cv2.cvtColor(input2, cv2.COLOR_BGR2GRAY)

  edges1 = canny_detection(gray_input1)
  edges2 = canny_detection(gray_input2)

  maskedroi1 = maskROI(edges1)
  maskedroi2 = maskROI(edges2)  

  lines1 = cv2.HoughLinesP(maskedroi1, RHO, THETA, cv2.getTrackbarPos('Threshold','Trackbars'), np.array([]), minLineLength=cv2.getTrackbarPos('MIN_LINE_LEN','Trackbars'),maxLineGap=cv2.getTrackbarPos('MAX_LINE_GAP','Trackbars'))
  
  lines2  = cv2.HoughLinesP(maskedroi2, RHO, THETA, cv2.getTrackbarPos('Threshold','Trackbars'), np.array([]), minLineLength=cv2.getTrackbarPos('MIN_LINE_LEN','Trackbars'),maxLineGap=cv2.getTrackbarPos('MAX_LINE_GAP','Trackbars'))

  lined_img1 = draw_lines(lines1, maskedroi1)
  lined_img2 = draw_lines(lines2, maskedroi2)

  lanesof1 = formulate_lanes(lines1, maskedroi1)
  lanesof2 = formulate_lanes(lines2, maskedroi2)
  
  lanes_img1 = draw_lines(lanesof1, input1)
  lanes_img2 = draw_lines(lanesof2, input2)
  final_image1 = cv2.addWeighted(input1, 0.9, lanes_img1, 1, 0)
  final_image2 = cv2.addWeighted(input2, 0.9, lanes_img2, 1, 0)
  cv2.imshow('output1',final_image1)
  cv2.imshow('output2',final_image2)
  k=cv2.waitKey(10)
  if k == ord('q'):
    break
cv2.destroyAllWindows()  

Q3 read the following video and apply lane line detection on it then show output
using opencv.

In [17]:
cap=cv2.VideoCapture('solidWhiteRight.mp4')

def hough_lines(img):
  lines = cv2.HoughLinesP(img, RHO, THETA, 10, np.array([]), minLineLength=7,maxLineGap=7)
  return lines

def canny_detection(img):
    blurred = cv2.GaussianBlur(img,(5,5),0)
    return cv2.Canny(blurred,169,0)

def draw_lines(lines, masked_edges):
    color = [243, 105, 14]
    thickness = 12
    lines_image = np.zeros((masked_edges.shape[0], masked_edges.shape[1], 3), dtype=np.uint8)
    for line in lines:
      for x1,y1,x2,y2 in line:
        cv2.line(lines_image, (x1, y1), (x2, y2), color, thickness)
    return(lines_image)

while cap.isOpened():
    stat, frame=cap.read()
    if stat==True:
        
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        edges = canny_detection(gray_frame)
        
        maskedroi = maskROI(edges)
        
        lines = hough_lines(maskedroi)

        lanes = formulate_lanes(lines, maskedroi)
  
        lanes_img = draw_lines(lanes, frame)

        frame = cv2.addWeighted(frame, 0.9, lanes_img, 1, 0)
        
        cv2.imshow("my_video",frame)
        k=cv2.waitKey(20)
        if k==ord('q'):
            break
    else:
        cv2.destroyAllWindows()
        break
cap.release()
cv2.destroyAllWindows()

